# Assignment 1: Vector Database Creation and Retrieval
## Day 6 Session 2 - RAG Fundamentals

**OBJECTIVE:** Create a vector database from a folder of documents and implement basic retrieval functionality.

**LEARNING GOALS:**
- Understand document loading with SimpleDirectoryReader
- Learn vector store setup with LanceDB
- Implement vector index creation
- Perform semantic search and retrieval

**DATASET:** Use the data folder in `Day_6/session_2/data/` which contains multiple file types

**INSTRUCTIONS:**
1. Complete each function by replacing the TODO comments with actual implementation
2. Run each cell after completing the function to test it
3. The answers can be found in the existing notebooks in the `llamaindex_rag/` folder


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install llama-index llama-index-vector-stores-lancedb llama-index-embeddings-huggingface

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-rcn8txqs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-rcn8txqs
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=925d3bea99280c0e2149fd820b75670667ee7687a0c82fd0ebfb401926e61971
  Stored in directory: /tmp/pip-ephem-wheel-cache-ge2olkky/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [1]:
# Import required libraries
import os
from pathlib import Path
from typing import List
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [2]:
# Configure LlamaIndex Settings (Using OpenRouter - No OpenAI API Key needed)
def setup_llamaindex_settings():
    """
    Configure LlamaIndex with local embeddings and OpenRouter for LLM.
    This assignment focuses on vector database operations, so we'll use local embeddings only.
    """
    # Check for OpenRouter API key (for future use, not needed for this basic assignment)
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        print("ℹ️  OPENROUTER_API_KEY not found - that's OK for this assignment!")
        print("   This assignment only uses local embeddings for vector operations.")

    # Configure local embeddings (no API key required)
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5",
        trust_remote_code=True
    )

    print("✅ LlamaIndex configured with local embeddings")
    print("   Using BAAI/bge-small-en-v1.5 for document embeddings")

# Setup the configuration
setup_llamaindex_settings()


ℹ️  OPENROUTER_API_KEY not found - that's OK for this assignment!
   This assignment only uses local embeddings for vector operations.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ LlamaIndex configured with local embeddings
   Using BAAI/bge-small-en-v1.5 for document embeddings


## 1. Document Loading Function

Complete the function below to load documents from a folder using `SimpleDirectoryReader`.

**Note:** This assignment uses local embeddings only - no OpenAI API key required! We're configured to use OpenRouter for future LLM operations.


In [7]:
def load_documents_from_folder(folder_path: str):
    """
    Load documents from a folder using SimpleDirectoryReader.

    TODO: Complete this function to load documents from the given folder path.
    HINT: Use SimpleDirectoryReader with recursive parameter to load all files

    Args:
        folder_path (str): Path to the folder containing documents

    Returns:
        List of documents loaded from the folder
    """
    # TODO: Create SimpleDirectoryReader instance
    reader = SimpleDirectoryReader(
        input_dir=folder_path,
        recursive=True
    )

    # TODO: Load and return documents
    documents = reader.load_data()
    return documents

    # PLACEHOLDER - Replace with actual implementation
    print(f"TODO: Load documents from {folder_path}")

# Test the function after you complete it
test_folder = "/content/drive/MyDrive/datafiles"
documents = load_documents_from_folder(test_folder)
print(f"Loaded {len(documents)} documents")


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 191MiB/s]


Loaded 123 documents


## 2. Vector Store Creation Function

Complete the function below to create a LanceDB vector store.


In [8]:
def create_vector_store(db_path: str = "./vectordb", table_name: str = "documents"):
    """
    Create a LanceDB vector store for storing document embeddings.

    TODO: Complete this function to create and configure a LanceDB vector store.
    HINT: Use LanceDBVectorStore with uri and table_name parameters

    Args:
        db_path (str): Path where the vector database will be stored
        table_name (str): Name of the table in the vector database

    Returns:
        LanceDBVectorStore: Configured vector store
    """
    # TODO: Create the directory if it doesn't exist
    Path(db_path).mkdir(parents=True, exist_ok=True)

    # TODO: Create vector store
    vector_store = LanceDBVectorStore(
        uri=db_path,
        table_name=table_name
    )
    return vector_store

    # PLACEHOLDER - Replace with actual implementation
    print(f"TODO: Create vector store at {db_path}")

# Test the function after you complete it
vector_store = create_vector_store("./assignment_vectordb")
print(f"Vector store created: {vector_store is not None}")

Vector store created: True


## 3. Vector Index Creation Function

Complete the function below to create a vector index from documents.


In [9]:
def create_vector_index(documents: List, vector_store):
    """
    Create a vector index from documents using the provided vector store.

    TODO: Complete this function to create a VectorStoreIndex from documents.
    HINT: Create StorageContext with vector_store, then use VectorStoreIndex.from_documents()

    Args:
        documents: List of documents to index
        vector_store: LanceDB vector store to use for storage

    Returns:
        VectorStoreIndex: The created vector index
    """
    # TODO: Create storage context with vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # TODO: Create index from documents
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )
    return index

    # PLACEHOLDER - Replace with actual implementation
    print(f"TODO: Create vector index from {len(documents)} documents")

# Test the function after you complete it (will only work after previous functions are completed)
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print(f"Vector index created: {index is not None}")
else:
    print("Complete previous functions first to test this one")


Parsing nodes:   0%|          | 0/123 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/139 [00:00<?, ?it/s]

Vector index created: True


## 4. Document Search Function

Complete the function below to search for relevant documents using the vector index.


In [10]:
def search_documents(index, query: str, top_k: int = 3):
    """
    Search for relevant documents using the vector index.

    TODO: Complete this function to perform semantic search on the index.
    HINT: Use index.as_retriever() with similarity_top_k parameter, then retrieve(query)

    Args:
        index: Vector index to search
        query (str): Search query
        top_k (int): Number of top results to return

    Returns:
        List of retrieved document nodes
    """
    # TODO: Create retriever from index
    retriever = index.as_retriever(similarity_top_k=top_k)

    # TODO: Retrieve documents for the query
    results = retriever.retrieve(query)

    return results

    test_query = "What are AI agents?"
    print(f"TODO: Search for '{query}' in index")
    return []

# Test the function after you complete it (will only work after all previous functions are completed)
if 'index' in locals() and index is not None:
    test_query = "What are AI agents?"
    results = search_documents(index, test_query, top_k=2)
    print(f"Found {len(results)} results for query: '{test_query}'")
    for i, result in enumerate(results, 1):
        print(f"Result {i}: {result.text[:100] if hasattr(result, 'text') else 'No text'}...")
else:
    print("Complete all previous functions first to test this one")





Found 2 results for query: 'What are AI agents?'
Result 1: THE LANDSCAPE OF EMERGING AI AGENT ARCHITECTURES
FOR REASONING , PLANNING , AND TOOL CALLING : A S U...
Result 2: task. In single agent patterns there is no feedback mechanism from other AI agents; however, there m...


## 5. Final Test - Complete Pipeline

Once you've completed all the functions above, run this cell to test the complete pipeline with multiple search queries.


In [11]:
# Final test of the complete pipeline
print("🚀 Testing Complete Vector Database Pipeline")
print("=" * 50)

# Re-run the complete pipeline to ensure everything works
data_folder = "/content/drive/MyDrive/datafiles"
vector_db_path = "./assignment_vectordb"

# Step 1: Load documents
print("\n📂 Step 1: Loading documents...")
documents = load_documents_from_folder(data_folder)
print(f"   Loaded {len(documents)} documents")

# Step 2: Create vector store
print("\n🗄️ Step 2: Creating vector store...")
vector_store = create_vector_store(vector_db_path)
print("   Vector store status:", "✅ Created" if vector_store else "❌ Failed")

# Step 3: Create vector index
print("\n🔗 Step 3: Creating vector index...")
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print("   Index status:", "✅ Created" if index else "❌ Failed")
else:
    index = None
    print("   ❌ Cannot create index - missing documents or vector store")

# Step 4: Test multiple search queries
print("\n🔍 Step 4: Testing search functionality...")
if index:
    search_queries = [
        "What are AI agents?",
        "How to evaluate agent performance?",
        "Italian recipes and cooking",
        "Financial analysis and investment"
    ]

    for query in search_queries:
        print(f"\n   🔎 Query: '{query}'")
        results = search_documents(index, query, top_k=2)

        if results:
            for i, result in enumerate(results, 1):
                text_preview = result.text[:100] if hasattr(result, 'text') else "No text available"
                score = f" (Score: {result.score:.4f})" if hasattr(result, 'score') else ""
                print(f"      {i}. {text_preview}...{score}")
        else:
            print("      No results found")
else:
    print("   ❌ Cannot test search - index not created")

print("\n" + "=" * 50)
print("🎯 Assignment Status:")
print(f"   Documents loaded: {'✅' if documents else '❌'}")
print(f"   Vector store created: {'✅' if vector_store else '❌'}")
print(f"   Index created: {'✅' if index else '❌'}")
print(f"   Search working: {'✅' if index else '❌'}")

if documents and vector_store and index:
    print("\n🎉 Congratulations! You've successfully completed the assignment!")
    print("   You've built a complete vector database with search functionality!")
else:
    print("\n📝 Please complete the TODO functions above to finish the assignment.")


🚀 Testing Complete Vector Database Pipeline

📂 Step 1: Loading documents...
   Loaded 123 documents

🗄️ Step 2: Creating vector store...
   Vector store status: ✅ Created

🔗 Step 3: Creating vector index...


Parsing nodes:   0%|          | 0/123 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/139 [00:00<?, ?it/s]

   Index status: ✅ Created

🔍 Step 4: Testing search functionality...

   🔎 Query: 'What are AI agents?'
      1. THE LANDSCAPE OF EMERGING AI AGENT ARCHITECTURES
FOR REASONING , PLANNING , AND TOOL CALLING : A S U... (Score: 0.6225)

   🔎 Query: 'How to evaluate agent performance?'
      1. steps, but the answers are limited to Yes/No responses [7]. As the industry continues to pivot towar... (Score: 0.6786)

   🔎 Query: 'Italian recipes and cooking'
      1. Spaghetti Carbonara, Italian, 20, Easy, Pasta, 450
Margherita Pizza, Italian, 45, Medium, Tomato, 32... (Score: 0.6413)

   🔎 Query: 'Financial analysis and investment'
      1. Stock, AAPL, Apple Inc, 10000, 12500, 25.0, Medium
Stock, GOOGL, Alphabet Inc, 8000, 9200, 15.0, Med... (Score: 0.5796)

🎯 Assignment Status:
   Documents loaded: ✅
   Vector store created: ✅
   Index created: ✅
   Search working: ✅

🎉 Congratulations! You've successfully completed the assignment!
   You've built a complete vector database with search fun